Soma total de vendas por dia, ordens com status de "Delivered" e estate = NY

In [0]:
%python
bronze_path = "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/bronze/"
silver_path = "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/silver/"
gold_path = "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/gold/"
resource_path = "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/resource/origem"
resource_path_volume = "/Volumes/bikestore/logistica/bikestore_resource/origem/"

In [0]:
%python
silver_map= {
    #"tmp_silver_custumer": f"{silver_path}/customer/",
    "tmp_silver_orders": f"{silver_path}/orders/",
    #"tmp_silver_product": f"{silver_path}/product/",
}
for view_name, path in silver_map.items():
    (spark.read.format("delta")
     .load(path)
     .createOrReplaceTempView(view_name))

In [0]:
SELECT * FROM tmp_silver_orders limit 5

In [0]:
SELECT
order_date,
-- state,
-- status,
ROUND(sum(total_sale),2) as total_sale
FROM tmp_silver_orders
WHERE state = "NY"
AND status = "Delivered"
AND order_date IS NOT NULL
GROUP BY order_date
--, state, status 
ORDER BY order_date

In [0]:
%python
df_gold_ny_sales = spark.sql("""
SELECT
order_date,
-- state,
-- status,
ROUND(sum(total_sale),2) as total_sale
FROM tmp_silver_orders
WHERE state = "NY"
AND status = "Delivered"
AND order_date IS NOT NULL
GROUP BY order_date
--, state, status 
ORDER BY order_date                          
                                   """)

# salvando em delta na Silver
df_gold_ny_sales.write.mode("overwrite").format("delta").save(f"{gold_path}/ny_sales")

In [0]:
CREATE TABLE IF NOT EXISTS bikestore.logistica.gold_sales_ny
LOCATION "abfss://uc-external-curso-databricks-vf@sacursodatabricksvf.dfs.core.windows.net/bikestore/gold/ny_sales"

In [0]:
SELECT * from bikestore.logistica.gold_sales_ny